# MNIST

In [4]:
from google.colab import files

# Only to get dataset from KAGGLE

#downlaod token (Kaggle > settings > create new Token)
# upload kaggle.json (token) - required for api
files.upload()
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Add Training / Test data to folder
!kaggle competitions download -c mnist-challenge-for-xai-proj-b
!unzip mnist-challenge-for-xai-proj-b.zip -d /content/

### Modify and test Hyperparameters

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from concurrent.futures import ThreadPoolExecutor


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class MNISTDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = f"{self.labels.iloc[idx, 0]:05d}.png"
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('L')
        label = self.labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label


class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 1,
                out_channels = 32,
                kernel_size = 3,
                stride=1,
                padding="same"
            ),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32,64,3,1,"same"),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.out = nn.Linear(64*7*7, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, 64*7*7)
        output = self.out(x)
        return torch.log_softmax(output, dim=1)



def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    model.to(device)
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        total_train = 0
        correct_train = 0
        train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Training)")
        for images, labels in train_progress_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            train_progress_bar.set_postfix(loss=loss.item(), accuracy=100. * correct_train / total_train)

        train_accuracy = 100 * correct_train / total_train
        train_accuracies.append(train_accuracy)

        # Validation Loop
        model.eval()
        total_val = 0
        correct_val = 0
        val_progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)")
        with torch.no_grad():
            for images, labels in val_progress_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                val_progress_bar.set_postfix(accuracy=100. * correct_val / total_val)

        val_accuracy = 100 * correct_val / total_val
        val_accuracies.append(val_accuracy)

    return train_accuracies, val_accuracies


# Data Preparation
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = MNISTDataset(csv_file='/content/Kaggle_MNIST/labels_train.csv',
                             root_dir='/content/Kaggle_MNIST/train',
                             transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])


# Hyperparameters
num_classes = 10
default_lr = 0.001
default_bs = 32
default_epoch = 15
learning_rates = [default_lr, 0.01, 0.1]
batch_sizes = [16, default_bs, 64]
num_epochs = [5, 10, default_epoch, 20]

# Testing Learning Rates
for lr in learning_rates:
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_loader = DataLoader(train_set, batch_size=default_bs, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=default_bs, shuffle=False)

    train_acc, val_acc = train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=default_epoch)
    print("Train Accuracies:", [round(acc, 3) for acc in train_acc])
    print("Validation Accuracies:", [round(acc, 3) for acc in val_acc])
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title(f'Learning Rate: {lr}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Testing Batch Sizes
for batch_size in batch_sizes:
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=default_lr)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    train_acc, val_acc = train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=default_epoch)
    print("Train Accuracies:", [round(acc, 3) for acc in train_acc])
    print("Validation Accuracies:", [round(acc, 3) for acc in val_acc])
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title(f'Batch Size: {batch_size}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Testing Different Numbers of Epochs
for epochs in num_epochs:
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=default_lr)
    train_loader = DataLoader(train_set, batch_size=default_bs, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=default_bs, shuffle=False)

    train_acc, val_acc = train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs)
    print("Train Accuracies:", [round(acc, 3) for acc in train_acc])
    print("Validation Accuracies:", [round(acc, 3) for acc in val_acc])
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title(f'Number of Epochs: {epochs}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

### Use the best choices for each test

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from concurrent.futures import ThreadPoolExecutor


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class MNISTDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = f"{self.labels.iloc[idx, 0]:05d}.png"
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('L')
        label = self.labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label



class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding="same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding="same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding="same"),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc_bn = nn.BatchNorm1d(256)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc_bn(self.fc1(x)))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)


# Hyperparameters
num_classes = 10
batch_size = 32
learning_rate = 0.001
num_epochs = 30

# Data Preparation
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = MNISTDataset(csv_file='/content/Kaggle_MNIST/labels_train.csv',
                             root_dir='/content/Kaggle_MNIST/train',
                             transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Model, Loss Function, Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Training Loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation Loop
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.4f}, Validation Accuracy: {100 * correct / total:.2f}%')

print("Training completed")



In [ ]:
class CustomMNISTTestDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.file_names = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.file_names[idx])
        image = Image.open(img_path).convert('L')
        if self.transform:
            image = self.transform(image)
        return image, self.file_names[idx]

test_dataset = CustomMNISTTestDataset("/content/Kaggle_MNIST/test", transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
predictions = []
with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu().numpy()
        predictions.extend(zip(file_names, predicted))



submission_df = pd.DataFrame(predictions, columns=['ID', 'Label'])
submission_df['ID'] = submission_df['ID'].apply(lambda x: x.split('.')[0])

# Sort the DataFrame
submission_df['ID'] = pd.to_numeric(submission_df['ID'])  # Convert to numeric for correct sorting
submission_df = submission_df.sort_values(by='ID')
submission_df['ID'] = submission_df['ID'].apply(lambda x: f"{x:05d}")  # Convert back to string with leading zeros

submission_df.to_csv('/content/submissionMNIST.csv', index=False)

print("Submission file created")